In [ ]:
import os
import pickle

pickle_file_location = '/home/jxl220096/data/hay/haystac_trial1/fix_Baseline_TA1_Trial_Train_Submission/preprocess/full_train_raw_X_y.pkl'

with open(pickle_file_location, 'rb') as f:
    X, y = pickle.load(f)

In [ ]:
for dataset in os.listdir('/home/jxl220096/data/hay/haystac_trial1/'):
    print(dataset)

In [ ]:
print("X", X)
print("y", y)

In [3]:
parquet_file_path = '/home/jxl220096/data/hay/new_format/trial2/losangeles/test_stops_valsplit/fis_0b64fb44-1130-6b15-6a1f-922a945832d8/3813.parquet'
temp_parquet = pd.read_parquet(parquet_file_path)
temp_parquet.sort_values(by=['time_start'], inplace=True)
temp_parquet

,agent_id,time_start,time_stop,geometry,unique_stop_point,global_stop_points
12159130,3813,2024-01-29 08:00:00,2024-01-29 16:00:50,POINT (-118.1963799 33.916399000000006),8116c99a-7970-4587-859a-999bb526d196,8f1dfdec-cde0-49f0-a512-92c24826646d
12159165,3813,2024-01-29 16:38:43,2024-01-29 23:00:35,POINT (-117.90089 33.924819899999996),5a711eef-6cbe-4bd7-b8ad-c84ae95d8c50,606c78d6-6781-4f0d-9f79-7f8e1e41e344
12159131,3813,2024-01-29 23:40:47,2024-01-30 15:15:04,POINT (-118.19638000000002 33.9163991),8116c99a-7970-4587-859a-999bb526d196,8f1dfdec-cde0-49f0-a512-92c24826646d
12159166,3813,2024-01-30 15:56:43,2024-01-30 23:56:48,POINT (-117.90089 33.92481999999999),5a711eef-6cbe-4bd7-b8ad-c84ae95d8c50,606c78d6-6781-4f0d-9f79-7f8e1e41e344
12159132,3813,2024-01-31 00:34:48,2024-01-31 15:14:28,POINT (-118.1963799 33.9163991),8116c99a-7970-4587-859a-999bb526d196,8f1dfdec-cde0-49f0-a512-92c24826646d
...,...,...,...,...,...,...
12159194,3813,2024-02-23 20:22:36,2024-02-23 21:44:27,POINT (-118.1943334 33.9031274),c345a2d2-7e8f-4ace-b0d5-15b24151ae4f,d77f7bb4-27fd-45cd-98be-0a82c9467ac0
12159163,3813,2024-02-23 21:47:24,2024-02-23 23:26:36,POINT (-118.19637949999999 33.91639959999999),8116c99a-7970-4587-859a-999bb526d196,8f1dfdec-cde0-49f0-a512-92c24826646d
12159190,3813,2024-02-24 00:08:43,2024-02-24 01:58:08,POINT (-117.90089 33.924819899999996),5a711eef-6cbe-4bd7-b8ad-c84ae95d8c50,606c78d6-6781-4f0d-9f79-7f8e1e41e344
12159164,3813,2024-02-24 02:36:50,2024-02-24 19:00:03,POINT (-118.1963799 33.916399000000006),8116c99a-7970-4587-859a-999bb526d196,8f1dfdec-cde0-49f0-a512-92c24826646d


In [4]:
df = temp_parquet
# Assuming there are 'depart_time' at one stop point and 'arrive_time' at the next stop point
df['depart_time'] = pd.to_datetime(df['time_stop'])
df['arrive_time'] = pd.to_datetime(df['time_start'])

# Calculate travel time in seconds between stop points
df['travel_time'] = (df['arrive_time'].shift(-1) - df['depart_time']).dt.total_seconds()

# Remove any negative travel times which may occur due to data errors or misalignments
df = df[df['travel_time'] >= 0]

# Aggregate travel times by day
df['date'] = df['depart_time'].dt.date
daily_travel_times = df.groupby('date')['travel_time'].sum().reset_index()

# Calculate mean and anomalies
mean_travel_time = daily_travel_times['travel_time'].mean()
daily_travel_times['anomaly'] = daily_travel_times['travel_time'] - mean_travel_time

# Identify top 3 days with the highest travel time anomalies
top_travel_time_anomalies = daily_travel_times.nlargest(3, 'anomaly')

NameError: name 'df' is not defined

In [ ]:
df = temp_parquet
# # Assuming there are 'depart_time' at one stop point and 'arrive_time' at the next stop point
df['depart_time'] = pd.to_datetime(df['time_stop'])
df['arrive_time'] = pd.to_datetime(df['time_start'])
df['travel_time'] = (df['arrive_time'].shift(-1) - df['depart_time']).dt.total_seconds()
df = df[df['travel_time'] >= 0]
df['date'] = df['depart_time'].dt.date
daily_travel_times = df.groupby('date')['travel_time'].sum().reset_index()

# Calculate mean and anomalies
mean_travel_time = daily_travel_times['travel_time'].mean()
daily_travel_times['anomaly'] = daily_travel_times['travel_time'] - mean_travel_time

# Identify top 3 days with the highest travel time anomalies
top_travel_time_anomalies = daily_travel_times.nlargest(3, 'anomaly')
top_travel_time_anomalies


# Deal with Validation Set

In [2]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import pickle

In [13]:
fis_folder = '/home/jxl220096/data/hay/new_format/trial2/losangeles/test_stops_valsplit'

## 1.1 get validation set

In [14]:
import numpy as np
def get_fis_group(fis_folder, num_subsample_group=None):
    group_agent_list = []
    group_list = os.listdir(fis_folder)
    if num_subsample_group is not None:
        num_group = num_subsample_group
    else:
        num_group = len(group_list)
    for i in range(num_group):
        temp_group_folder = group_list[i]
        temp_parquet_list = os.listdir(os.path.join(fis_folder, temp_group_folder))
        temp_agent_list = [np.int64(temp_file.replace('.parquet', '')) for temp_file in temp_parquet_list]
        group_agent_list.append(temp_agent_list)
    return group_agent_list

group_agent_list = get_fis_group(fis_folder)

len(group_agent_list)
print(group_agent_list[0])

[263164, 16350, 168094, 125182, 283458, 120354, 190858, 176486, 63509, 234476, 26260, 196774, 143175, 176002, 200976, 89105, 50833, 100303, 171606, 87261, 181238, 259614, 260858, 132918, 4499, 188754, 59965, 180100, 155995, 154758, 215095, 101798, 13567, 153239, 249207, 57288, 15319, 112727, 268326, 216149, 51213, 196879, 45558, 138907, 241265, 238941, 236863, 228761, 263710, 254392, 168289, 104291, 208253, 22306, 262162, 122629, 256229, 118117, 81544, 155390, 259279, 109163, 55133, 193590, 258959, 137152, 252999, 128753, 114858, 288802, 159609, 118824, 290529, 201305, 29676, 43118, 49951, 221594, 240630, 239380, 119190, 76612, 134012, 199816, 108405, 166118, 240192, 41980, 98169, 242038, 26178, 189480, 284812, 184025, 289042, 196899, 44142, 191243, 193695, 96364, 235190, 94922, 85432, 203626, 284586, 120482, 76777, 256396, 170426, 148580, 29813, 92259, 205825, 276902, 130042, 75778, 171739, 154866, 205758, 225728, 277842, 255280, 44591, 154977, 223185, 80560, 230700, 51997, 227430, 28

## 1.2 Calculate features of agents under validation set

### [1] Feature: Travel Distance Anomaly

In [16]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

def travel_distance_anomaly(df, normalize_method):
    """
    Calculate the top 3 travel distance anomalies for an agent's daily distances.
    This function computes the Euclidean distances between consecutive geographic points recorded for an agent,
    aggregates these distances by day, calculates the mean daily distance, and identifies the days
    with the top three highest deviations from this mean.

    Parameters:
    - df (pd.DataFrame): A DataFrame containing the agent's data with columns 'geometry', 'time_start'.
    - normalize_method (str): A string to select the normalization method; currently supports 'wo_std'.

    Returns:
    - pd.DataFrame: A DataFrame containing the dates and distance anomalies of the top three anomalies.
    """


    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Calculate Euclidean Distances
    gdf.sort_values(by=['time_start'], inplace=True)
    gdf['distance'] = gdf['geometry'].shift(-1).distance(gdf['geometry'])
    gdf['distance'] = gdf['distance'].fillna(0)

    # Aggregate Distances by Day
    gdf['date'] = pd.to_datetime(gdf['time_start']).dt.date
    daily_distances = gdf.groupby('date')['distance'].sum().reset_index()

    # Calculate Mean and Anomalies
    mean_distance = daily_distances['distance'].mean()
    if normalize_method == 'wo_std':
        daily_distances['anomaly'] = daily_distances['distance'] - mean_distance
    top_travel_distance_anomaly = daily_distances.nlargest(3, 'anomaly')
    
    return top_travel_distance_anomaly

parquet_file_path = '/home/jxl220096/data/hay/new_format/trial2/losangeles/test_stops_valsplit/fis_0b64fb44-1130-6b15-6a1f-922a945832d8/3813.parquet'
temp_parquet = pd.read_parquet(parquet_file_path)
travel_distance_anomaly(temp_parquet, 'wo_std')

,date,distance,anomaly
24,2024-02-22,1.227072,0.624814
18,2024-02-16,1.227071,0.624813
4,2024-02-02,0.941562,0.339303


### [2] Feature: travel_time_anomaly

In [17]:
def travel_time_anomaly(df, normalize_method):
    """
    Calculate the top 3 travel time anomalies for an agent's daily travel times.
    This function computes the total travel time for each day, calculates the mean daily travel time, and identifies
    the days with the top three highest deviations from this mean travel time.

    Parameters:
    - df (pd.DataFrame): A DataFrame containing the agent's data with columns 'time_start' and 'time_stop'.

    Returns:
    - pd.DataFrame: A DataFrame containing the dates and travel time anomalies of the top three anomalies.
    """
    
    # Ensure time_start and time_stop are datetime
    df['arrive_time'] = pd.to_datetime(df['time_start'])
    df['depart_time'] = pd.to_datetime(df['time_stop'])

    # Calculate travel time in seconds
    df['travel_time'] = (df['arrive_time'].shift(-1) - df['depart_time']).dt.total_seconds()
    df = df[df['travel_time'] >= 0]
    
    # Aggregate travel times by day
    df['date'] = df['arrive_time'].dt.date
    daily_travel_times = df.groupby('date')['travel_time'].sum().reset_index()

    # Calculate mean and anomalies
    mean_travel_time = daily_travel_times['travel_time'].mean()
    if normalize_method == 'wo_std':
        daily_travel_times['anomaly'] = daily_travel_times['travel_time'] - mean_travel_time
    top_travel_time_anomaly = daily_travel_times.nlargest(3, 'anomaly')

    return top_travel_time_anomaly

parquet_file_path = '/home/jxl220096/data/hay/new_format/trial2/losangeles/test_stops_valsplit/fis_0b64fb44-1130-6b15-6a1f-922a945832d8/3813.parquet'
temp_parquet = pd.read_parquet(parquet_file_path)
travel_time_anomaly(temp_parquet, 'wo_std')

/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['arrive_time'].dt.date


,date,travel_time,anomaly
8,2024-02-06,2974073.0,2.622123e+06
5,2024-02-03,1520678.0,1.168728e+06
19,2024-02-17,626348.0,2.743978e+05


### [3] Feature: unique_locations_anomaly

In [18]:
def unique_locations_anomaly(df, normalize_method):
    """
    Calculate the daily unique locations visited by an agent and identify the deviation from the mean unique locations per day.
    This function counts the unique stop points visited each day, computes the mean of these counts, and finds the daily deviations.

    Parameters:
    - df (pd.DataFrame): A DataFrame containing the agent's data with columns 'time_start' and 'unique_stop_point'.

    Returns:
    - pd.DataFrame: A DataFrame containing the dates, number of unique locations visited, and the deviations from the mean count of unique locations.
    """

    # Ensure time_start is datetime
    df['time_start'] = pd.to_datetime(df['time_start'])

    # Extract the date from time_start
    df['date'] = df['time_start'].dt.date

    # Group by date and count unique stop points
    daily_unique_locations = df.groupby('date')['unique_stop_point'].nunique().reset_index()
    daily_unique_locations.rename(columns={'unique_stop_point': 'unique_locations'}, inplace=True)

    # Calculate the mean of unique locations visited
    mean_unique_locations = daily_unique_locations['unique_locations'].mean()

    # Calculate deviations from the mean
    if normalize_method == 'wo_std':
        daily_unique_locations['anomaly'] = daily_unique_locations['unique_locations'] - mean_unique_locations
    top_travel_time_anomaly = daily_unique_locations.nlargest(3, 'anomaly')

    return top_travel_time_anomaly

parquet_file_path = '/home/jxl220096/data/hay/new_format/trial2/losangeles/test_stops_valsplit/fis_0b64fb44-1130-6b15-6a1f-922a945832d8/3813.parquet'
temp_parquet = pd.read_parquet(parquet_file_path)
unique_locations_anomaly(temp_parquet, 'wo_std')

,date,unique_locations,anomaly
4,2024-02-02,4,1.555556
8,2024-02-06,4,1.555556
25,2024-02-23,4,1.555556


### [4] Feature: Duration Anomalies

In [19]:
import pandas as pd

def duration_anomalies(df, normalize_method):
    """
    Calculate the top 3 highest anomalies in durations spent daily compared to the mean duration for an agent.
    This function computes the total duration spent each day, calculates the mean duration, and finds the days
    with the top three highest deviations from this mean duration.

    Parameters:
    - df (pd.DataFrame): A DataFrame containing the agent's data with columns 'time_start' and 'time_stop'.

    Returns:
    - pd.DataFrame: A DataFrame containing the dates and the duration anomalies of the top three anomalies.
    """

    # Ensure time_start and time_stop are datetime
    df['time_start'] = pd.to_datetime(df['time_start'])
    df['time_stop'] = pd.to_datetime(df['time_stop'])

    # Calculate duration in seconds
    df['duration'] = (df['time_stop'] - df['time_start']).dt.total_seconds()

    # Aggregate durations by day
    df['date'] = df['time_start'].dt.date
    daily_durations = df.groupby('date')['duration'].sum().reset_index()

    # Calculate mean and anomalies
    mean_duration = daily_durations['duration'].mean()
    if normalize_method == 'wo_std':
        daily_durations['anomaly'] = daily_durations['duration'] - mean_duration

    # Identify top 3 days with the highest duration anomalies
    top_duration_anomalies = daily_durations.nlargest(3, 'anomaly')

    return top_duration_anomalies

parquet_file_path = '/home/jxl220096/data/hay/new_format/trial2/losangeles/test_stops_valsplit/fis_0b64fb44-1130-6b15-6a1f-922a945832d8/3813.parquet'
temp_parquet = pd.read_parquet(parquet_file_path)
duration_anomalies(temp_parquet, 'wo_std')

,date,duration,anomaly
21,2024-02-19,132503.0,52943.666667
13,2024-02-11,127026.0,47466.666667
0,2024-01-29,107819.0,28259.666667


## 1.5 process_multiple_parquets

In [20]:
def process_multiple_parquets(folder_path, file_list):
    """
    Process multiple parquet files to find travel distance anomalies for each agent.

    Parameters:
    - file_list (list): A list of strings, where each string is the file path to a parquet file.

    Returns:
    - dict: A dictionary where each key is the filename (assuming it's unique per agent) 
            and the value is the DataFrame of top three anomalies.
    """
    feature_dict = {}
    for item in file_list:
        file_path = os.path.join(folder_path, item)
        df = pd.read_parquet(file_path)
        agent_id = os.path.splitext(os.path.basename(file_path))[0]  # Assuming file name is the agent_id

        top3_travel_distance_anomaly = list(travel_distance_anomaly(df, 'wo_std')['anomaly'])
        top3_travel_time_anomaly = list(travel_time_anomaly(df, 'wo_std')['anomaly'])
        top3_unique_locations_anomaly = list(unique_locations_anomaly(df, 'wo_std')['anomaly'])
        top3_duration_anomalies = list(duration_anomalies(temp_parquet, 'wo_std')['anomaly'])
        
        features = top3_travel_distance_anomaly + top3_travel_time_anomaly + top3_unique_locations_anomaly + top3_duration_anomalies
        feature_dict[agent_id] = features
    return feature_dict

parquet_group_1 = '/home/jxl220096/data/hay/new_format/trial2/losangeles/test_stops_valsplit/fis_0b64fb44-1130-6b15-6a1f-922a945832d8/'
parquet_files = os.listdir(parquet_group_1)
feature_dict = process_multiple_parquets(parquet_group_1, parquet_files)
print(feature_dict['170636'])

/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['arrive_time'].dt.date
/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['arrive_time'].dt.date
/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

[0.17034046987179324, 0.0888869817944011, 0.07604898899458429, 1064701.4642857143, 704039.4642857143, 636798.4642857143, 1.1724137931034482, 0.17241379310344818, 0.17241379310344818, 52943.66666666667, 47466.66666666667, 28259.66666666667]


/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['arrive_time'].dt.date
/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['arrive_time'].dt.date
/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [22]:
def get_group_feature_list(fis_folder, num_subsample_group=None):
    group_feature_list = []
    group_agent_list = []
    group_list = os.listdir(fis_folder)
    if num_subsample_group is not None:
        num_group = num_subsample_group
    else:
        num_group = len(group_list)
    for i in range(num_group):
        temp_group_folder = group_list[i]
        temp_parquet_list = os.listdir(os.path.join(fis_folder, temp_group_folder))
        temp_feature_dict = process_multiple_parquets(os.path.join(fis_folder, temp_group_folder), temp_parquet_list)
        temp_agent_list = [np.int64(temp_file.replace('.parquet', '')) for temp_file in temp_parquet_list]
        
        group_feature_list.append(temp_feature_dict)
        group_agent_list.append(temp_agent_list)

    return group_feature_list, group_agent_list

fis_folder = '/home/jxl220096/data/hay/new_format/trial2/losangeles/test_stops_valsplit/'
group_feature_list , group_agent_list= get_group_feature_list(fis_folder, num_subsample_group=1)

/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['arrive_time'].dt.date
/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['arrive_time'].dt.date
/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

## 1.4 Get labels

In [ ]:
print(group_feature_list)
print( len (list(  (group_feature_list[0].keys()))))

In [23]:
import pickle
# If the agent in abnormal_agent_id_list, then the agent's label is 1, else 0.
train_dataset_folder = '/home/jxl220096/data/hay/new_format/trial2/losangeles/train_stops/'
agent_anomaly_days_dict = np.load(os.path.join(train_dataset_folder, "preprocess/agent_anomaly_days_dict.npy"), allow_pickle=True).item()
abnormal_agent_id_list = list(agent_anomaly_days_dict.keys())

X = []
y = []

for group in group_feature_list:
    for agent_id, features in group.items():
        X.append(features) 
        if agent_id in abnormal_agent_id_list:
            y.append(1)  
        else:
            y.append(0)  

with open('wo_std_X_y.pkl', 'wb') as f:
    pickle.dump((X, y), f)

## 1.5 split validation set at first

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train)
print(len(X_train))
print(len(X_train[0]))
print(X_test)
print(len(X_test))
print(len(X_test[0]))


[[0.4204599492562129, 0.26859577012116487, 0.1449003083274345, 773552.3846153846, 667735.3846153846, 619458.3846153846, 1.5555555555555554, 1.5555555555555554, 0.5555555555555554, 52943.66666666667, 47466.66666666667, 28259.66666666667], [0.9433799850579878, 0.34680348251859483, 0.29426228907747265, 615349.44, 537185.44, 368081.44, 3.6296296296296298, 1.6296296296296298, 0.6296296296296298, 52943.66666666667, 47466.66666666667, 28259.66666666667], [0.1246953665973344, 0.10752515378558278, 0.1061349333164818, 1163748.12, 545885.12, 535212.12, 1.5384615384615383, 1.5384615384615383, 1.5384615384615383, 52943.66666666667, 47466.66666666667, 28259.66666666667], [0.5208850066445195, 0.4888742898710031, 0.47307257602655284, 2181571.703703704, 705630.7037037037, 463835.7037037037, 1.25, 1.25, 1.25, 52943.66666666667, 47466.66666666667, 28259.66666666667], [0.4032086242299258, 0.38737634623544676, 0.35790818839567745, 902874.6, 443536.6, 399471.6, 2.0357142857142856, 1.0357142857142858, 1.0357

# 2. Deal with whole training set

In [25]:
import os
import pandas as pd

train_dataset_folder = '/home/jxl220096/data/hay/new_format/trial2/losangeles/train_stops/'
parquet_path = os.path.join(train_dataset_folder, '80107.parquet')
temp_parquet = pd.read_parquet(parquet_path)

travel_time_anomaly(temp_parquet, 'wo_std')

/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['arrive_time'].dt.date


,date,travel_time,anomaly
6,2024-01-07,1642952.0,1.269897e+06
11,2024-01-12,1615097.0,1.242042e+06
15,2024-01-16,1591386.0,1.218331e+06


In [ ]:
def find_parquet_files(directory):
    """
    Find and return a list of full paths to .parquet files in the given directory.

    :param directory: The path to the directory to search.
    :return: A list of full paths to .parquet files.
    """
    parquet_files = []
    # Walk through all files and folders within the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".parquet"):
                full_path = os.path.join(root, file)
                parquet_files.append(full_path)
    return parquet_files

In [26]:

def preprocess_training_set_features(train_dataset_folder):
    train_df_list = os.listdir(train_dataset_folder)
    train_feature_dict = process_multiple_parquets(train_dataset_folder, train_df_list)
    train_agent_list = [np.int64(temp_file.replace('.parquet', '')) for temp_file in train_df_list]

    print(train_feature_dict.keys())

preprocess_training_set_features(train_dataset_folder)

/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['arrive_time'].dt.date
/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['arrive_time'].dt.date
/tmp/ipykernel_1449936/1975146511.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.